In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
# import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
from adastab1 import AdaStab
from OurAdam import Adam
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.io as sio

In [2]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

In [3]:
batch_size = 128
learning_rate = 1e-4
epochs = 10

In [4]:
transform_train = transforms.Compose([
    # transforms.RandomCrop(32, padding=4),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, 
                                       transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
def cifartest():
        correct = 0
        total = torch.FloatTensor([0])
#     with torch.no_grad():
        test_loss = 0
        
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = Variable(inputs, volatile=True),Variable(targets, volatile=True)
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss
            
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += (predicted==targets).sum()
            
        print('test loss:{}'.format(test_loss/(batch_idx+1)))
        
        
#         print(total)
#         print(correct)
#         total = Variable(total, requires_grad=False)
#         total = total.cuda()
        correct = correct.data[0]
#         correct = torch.FloatTensor(correct)
#         correct = Varaible(correct, requires_grad=False)
#         correct = correct.cuda()
#         correct = torch.Tensor(correct)
        acc = correct/total
        print('Accuracy of the network on the 10000 test images: %f' % (
         acc))
        
        return test_loss/(batch_idx+1), acc

In [6]:
def cifartrain():
    for epoch in range(max_epoch):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
        # get the inputs
            inputs, labels = data
            inputs = Variable(inputs)
            labels = Variable(labels)
            inputs = inputs.cuda()
            labels = labels.cuda()

        # zero the parameter gradients
            optimizer.zero_grad()

        # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss
        
        print('epoch{}, loss{}'.format(epoch, running_loss/(i+1)))
        LOSS[0, epoch] = running_loss/(i+1)
        # Test every epoch
        Test_LOSS[0, epoch], TEST_Acc[0, epoch] = cifartest()
    
    print('Finished Training')
    return LOSS, Test_LOSS, TEST_Acc

In [31]:
max_epoch=10
# No test

NosAdam

In [32]:
learning_rate = 5e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.3, gamma=0.05)
LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 1.5630
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2400
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.024000
epoch1, lossVariable containing:
 1.0865
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.0015
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.013900
epoch2, lossVariable containing:
 0.8787
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8637
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.025100
epoch3, lossVariable containing:
 0.7599
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.7983
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.018500
epoch4, lossVariable containing:
 0.6836
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

In [34]:
max_epoch=50

In [35]:
# learning_rate = 5e-5
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# TEST_Acc = np.zeros([1, max_epoch])
# optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.4, gamma=0.05)
# LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 1.5871
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2675
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.011100
epoch1, lossVariable containing:
 1.1124
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.9872
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.003700
epoch2, lossVariable containing:
 0.9082
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.9042
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.009900
epoch3, lossVariable containing:
 0.7912
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8172
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.014600
epoch4, lossVariable containing:
 0.7137
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

epoch36, lossVariable containing:
 0.4896
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.6964
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.012500
epoch37, lossVariable containing:
 0.4952
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.6931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.016500
epoch38, lossVariable containing:
 0.5034
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.6976
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.018700
epoch39, lossVariable containing:
 0.5127
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.7167
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.014500
epoch40, lossVariable containing:
 0.5306
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

KeyboardInterrupt: 

In [36]:
learning_rate = 5e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.5, gamma=0.05)
LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 1.6229
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3053
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.023800
epoch1, lossVariable containing:
 1.1517
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.0826
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.001800
epoch2, lossVariable containing:
 0.9641
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.9222
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.007000
epoch3, lossVariable containing:
 0.8400
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8471
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.004300
epoch4, lossVariable containing:
 0.7599
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

epoch36, lossVariable containing:
 0.6003
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.7469
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.017000
epoch37, lossVariable containing:
 0.6116
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.7609
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.016300
epoch38, lossVariable containing:
 0.6226
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.7934
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.007400
epoch39, lossVariable containing:
 0.6427
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8023
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.008600
epoch40, lossVariable containing:
 0.6549
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

In [37]:
sio.savemat('nosadam_res_gen5.mat',{'loss':LOSS, 'test_loss':Test_LOSS, 'test_acc':Test_Acc})

In [38]:
learning_rate = 5e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.6, gamma=0.05)
LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 1.6394
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3187
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.015100
epoch1, lossVariable containing:
 1.1907
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.0766
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.000800
epoch2, lossVariable containing:
 0.9893
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.9969
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.011300
epoch3, lossVariable containing:
 0.8659
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8599
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.006100
epoch4, lossVariable containing:
 0.7774
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

epoch36, lossVariable containing:
 0.6568
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.7964
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.007500
epoch37, lossVariable containing:
 0.6697
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8098
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.008700
epoch38, lossVariable containing:
 0.6871
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8239
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.008700
epoch39, lossVariable containing:
 0.6981
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8274
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.015300
epoch40, lossVariable containing:
 0.7118
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

In [39]:
sio.savemat('nosadam_res_gen6.mat',{'loss':LOSS, 'test_loss':Test_LOSS, 'test_acc':Test_Acc})

In [40]:
# learning_rate = 5e-6
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# TEST_Acc = np.zeros([1, max_epoch])
# optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.3, gamma=0.05)
# LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 2.1154
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.9221
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.002800
epoch1, lossVariable containing:
 1.8014
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.6924
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.008900
epoch2, lossVariable containing:
 1.5977
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.5264
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.011100
epoch3, lossVariable containing:
 1.4419
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.4004
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.007800
epoch4, lossVariable containing:
 1.3110
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

KeyboardInterrupt: 

In [41]:
learning_rate = 1e-6
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.2, gamma=0.05)
LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 2.3065
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2737
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.001400
epoch1, lossVariable containing:
 2.2417
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2061
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.020300
epoch2, lossVariable containing:
 2.1502
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.0866
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.011100
epoch3, lossVariable containing:
 2.0250
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.9696
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.015800
epoch4, lossVariable containing:
 1.9161
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

epoch36, lossVariable containing:
 0.6910
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1725
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.023600
epoch37, lossVariable containing:
 0.6565
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1793
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.024700
epoch38, lossVariable containing:
 0.6220
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1888
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.020500
epoch39, lossVariable containing:
 0.5862
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1923
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.025100
epoch40, lossVariable containing:
 0.5529
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

In [42]:
learning_rate = 1e-6
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.3, gamma=0.05)
LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 2.2790
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2483
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.023100
epoch1, lossVariable containing:
 2.2182
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.1830
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.016300
epoch2, lossVariable containing:
 2.1383
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.0899
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.015500
epoch3, lossVariable containing:
 2.0523
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.0183
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.013700
epoch4, lossVariable containing:
 1.9874
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

epoch36, lossVariable containing:
 0.7940
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1662
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.004000
epoch37, lossVariable containing:
 0.7594
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1726
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.004100
epoch38, lossVariable containing:
 0.7224
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1759
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.006300
epoch39, lossVariable containing:
 0.6850
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1788
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.007300
epoch40, lossVariable containing:
 0.6491
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

In [43]:
sio.savemat('nosdam_res_gen.mat',{'loss':LOSS, 'test_loss':Test_LOSS, 'test_acc':Test_Acc})

In [44]:
learning_rate = 8e-7
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.3, gamma=0.05)
LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 2.2775
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2451
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.022900
epoch1, lossVariable containing:
 2.2198
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.1913
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.003200
epoch2, lossVariable containing:
 2.1642
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.1326
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.004000
epoch3, lossVariable containing:
 2.0981
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.0609
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.002900
epoch4, lossVariable containing:
 2.0218
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

epoch36, lossVariable containing:
 1.0349
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2087
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.008200
epoch37, lossVariable containing:
 1.0074
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2013
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.013700
epoch38, lossVariable containing:
 0.9806
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1962
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.015800
epoch39, lossVariable containing:
 0.9561
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1900
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.016300
epoch40, lossVariable containing:
 0.9259
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

In [45]:
sio.savemat('nosadam_res_gen.mat',{'loss':LOSS, 'test_loss':Test_LOSS, 'test_acc':Test_Acc})

In [7]:
max_epoch=100

In [48]:
# learning_rate = 9e-7
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# TEST_Acc = np.zeros([1, max_epoch])
# optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.3, gamma=0.05)
# LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 2.2931
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2655
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.013000
epoch1, lossVariable containing:
 2.2342
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2049
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.024800
epoch2, lossVariable containing:
 2.1655
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.1256
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.004300
epoch3, lossVariable containing:
 2.0785
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.0359
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.016700
epoch4, lossVariable containing:
 1.9925
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

epoch36, lossVariable containing:
 0.9235
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2238
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.003500
epoch37, lossVariable containing:
 0.8909
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2166
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.007400
epoch38, lossVariable containing:
 0.8575
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2129
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.010400
epoch39, lossVariable containing:
 0.8249
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2214
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.007900
epoch40, lossVariable containing:
 0.7920
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

KeyboardInterrupt: 

In [54]:
learning_rate = 8e-7
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.3, gamma=0.05)
LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 2.2850
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2598
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.019500
epoch1, lossVariable containing:
 2.2336
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2061
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.023200
epoch2, lossVariable containing:
 2.1735
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.1362
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.001200
epoch3, lossVariable containing:
 2.0993
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.0607
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.001800
epoch4, lossVariable containing:
 2.0293
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

epoch36, lossVariable containing:
 1.0434
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2252
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.002800
epoch37, lossVariable containing:
 1.0167
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2160
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.009300
epoch38, lossVariable containing:
 0.9929
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2088
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.012000
epoch39, lossVariable containing:
 0.9632
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2039
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.015200
epoch40, lossVariable containing:
 0.9369
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

epoch72, lossVariable containing:
 0.2177
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3479
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.022700
epoch73, lossVariable containing:
 0.2094
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3433
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.001300
epoch74, lossVariable containing:
 0.2054
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3522
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.022700
epoch75, lossVariable containing:
 0.1955
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3523
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.025200
epoch76, lossVariable containing:
 0.1880
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

In [55]:
sio.savemat('nosadam_res_gen8.mat',{'loss':LOSS, 'test_loss':Test_LOSS, 'test_acc':Test_Acc})

In [52]:
learning_rate = 7e-7
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.3, gamma=0.05)
LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 2.2925
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2668
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.006600
epoch1, lossVariable containing:
 2.2425
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2180
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.011200
epoch2, lossVariable containing:
 2.1882
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.1573
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.021900
epoch3, lossVariable containing:
 2.1165
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.0754
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.004600
epoch4, lossVariable containing:
 2.0337
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

epoch36, lossVariable containing:
 1.1804
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2983
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.003300
epoch37, lossVariable containing:
 1.1562
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2884
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.006200
epoch38, lossVariable containing:
 1.1330
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2809
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.009300
epoch39, lossVariable containing:
 1.1114
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2747
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.010000
epoch40, lossVariable containing:
 1.0862
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

epoch72, lossVariable containing:
 0.3236
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3313
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.021800
epoch73, lossVariable containing:
 0.3090
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3351
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.024200
epoch74, lossVariable containing:
 0.2983
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3399
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.024800
epoch75, lossVariable containing:
 0.2847
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3443
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.023200
epoch76, lossVariable containing:
 0.2679
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

In [53]:
sio.savemat('nosadam_res_gen7.mat',{'loss':LOSS, 'test_loss':Test_LOSS, 'test_acc':Test_Acc})

In [8]:
learning_rate = 7e-7
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.4, gamma=0.05)
LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 2.3416
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.3240
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.010400
epoch1, lossVariable containing:
 2.3089
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2929
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.019700
epoch2, lossVariable containing:
 2.2782
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2612
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.003700
epoch3, lossVariable containing:
 2.2425
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2213
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.014100
epoch4, lossVariable containing:
 2.1937
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

epoch36, lossVariable containing:
 1.2166
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3029
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.025100
epoch37, lossVariable containing:
 1.1960
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2942
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.005900
epoch38, lossVariable containing:
 1.1728
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2827
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.007200
epoch39, lossVariable containing:
 1.1525
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2740
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.009700
epoch40, lossVariable containing:
 1.1291
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

epoch72, lossVariable containing:
 0.3420
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3170
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.020600
epoch73, lossVariable containing:
 0.3252
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3136
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.020600
epoch74, lossVariable containing:
 0.3113
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3316
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.016400
epoch75, lossVariable containing:
 0.2986
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3267
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.014600
epoch76, lossVariable containing:
 0.2840
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

In [9]:
sio.savemat('nosadam_res_gen77.mat',{'loss':LOSS, 'test_loss':Test_LOSS, 'test_acc':Test_Acc})

In [10]:
learning_rate = 8e-7
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, weight_decay=0.4, gamma=0.05)
LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 2.3058
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2876
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.000600
epoch1, lossVariable containing:
 2.2645
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.2450
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.009800
epoch2, lossVariable containing:
 2.2188
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.1937
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.023200
epoch3, lossVariable containing:
 2.1584
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 2.1215
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.006400
epoch4, lossVariable containing:
 2.0817
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

epoch36, lossVariable containing:
 1.0955
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2432
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.021500
epoch37, lossVariable containing:
 1.0691
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2373
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.022500
epoch38, lossVariable containing:
 1.0463
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2291
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.002100
epoch39, lossVariable containing:
 1.0174
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2225
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.003400
epoch40, lossVariable containing:
 0.9928
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

epoch72, lossVariable containing:
 0.2266
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3341
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.023600
epoch73, lossVariable containing:
 0.2184
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3456
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.022000
epoch74, lossVariable containing:
 0.2110
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3559
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.022100
epoch75, lossVariable containing:
 0.2013
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.3464
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.001400
epoch76, lossVariable containing:
 0.1981
[torch.cuda.FloatTensor of size 1 (GPU 0)]

te

In [11]:
sio.savemat('nosadam_res_gen88.mat',{'loss':LOSS, 'test_loss':Test_LOSS, 'test_acc':Test_Acc})

In [12]:
learning_rate = 8e-7
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.3,momentum=0.9)
LOSS, Test_LOSS, Test_Acc= cifartrain()

NameError: name 'model' is not defined

### AMSGrad

In [15]:
# learning_rate = 7e-5
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# optimizer = Adam(net.parameters(), lr=learning_rate, amsgrad=True)
# LOSS, Test_LOSS= cifartrain()

In [21]:
# learning_rate = 6e-5
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# TEST_Acc = np.zeros([1, max_epoch])
# optimizer = Adam(net.parameters(), lr=learning_rate, weight_decay=0.1, amsgrad=True)
# LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 1.4267
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1588
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.015400
epoch1, lossVariable containing:
 0.9894
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.9523
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.002600
epoch2, lossVariable containing:
 0.7920
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8006
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.012700
epoch3, lossVariable containing:
 0.6621
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.7656
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.023700
epoch4, lossVariable containing:
 0.5738
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

In [22]:
learning_rate = 6e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = Adam(net.parameters(), lr=learning_rate, weight_decay=0.2, amsgrad=True)
LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 1.4708
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1770
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.018900
epoch1, lossVariable containing:
 1.0185
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.9832
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.022200
epoch2, lossVariable containing:
 0.8357
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8277
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.005800
epoch3, lossVariable containing:
 0.7187
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.7435
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.011400
epoch4, lossVariable containing:
 0.6420
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

In [24]:
learning_rate = 6e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = Adam(net.parameters(), lr=learning_rate, weight_decay=0.3, amsgrad=True)
LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 1.5183
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2324
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.021400
epoch1, lossVariable containing:
 1.0536
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.9876
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.022000
epoch2, lossVariable containing:
 0.8667
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8873
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.014200
epoch3, lossVariable containing:
 0.7541
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.7527
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.011700
epoch4, lossVariable containing:
 0.6842
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

In [8]:
# learning_rate = 6e-5
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# TEST_Acc = np.zeros([1, max_epoch])
# optimizer = Adam(net.parameters(), lr=learning_rate, weight_decay=0.4, amsgrad=True)
# LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 1.5290
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2579
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.021000
epoch1, lossVariable containing:
 1.0941
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.0165
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.012900
epoch2, lossVariable containing:
 0.9148
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8588
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.000600
epoch3, lossVariable containing:
 0.7988
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8053
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.001500
epoch4, lossVariable containing:
 0.7268
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

In [ ]:
# learning_rate = 1e-4
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# TEST_Acc = np.zeros([1, max_epoch])
# optimizer = Adam(net.parameters(), lr=learning_rate, weight_decay=0.3, amsgrad=True)
# LOSS, Test_LOSS, Test_Acc= cifartrain()

epoch0, lossVariable containing:
 1.4528
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.1747
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.015200
epoch1, lossVariable containing:
 1.0529
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.9826
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.002500
epoch2, lossVariable containing:
 0.8919
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8673
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.000600
epoch3, lossVariable containing:
 0.7935
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8222
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.010400
epoch4, lossVariable containing:
 0.7463
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

In [30]:
learning_rate = 5e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
optimizer = Adam(net.parameters(), lr=learning_rate, amsgrad=True)
LOSS, Test_LOSS= cifartrain()

KeyboardInterrupt: 

In [14]:
# learning_rate = 4e-5
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# optimizer = Adam(net.parameters(), lr=learning_rate, amsgrad=True)
# LOSS, Test_LOSS= cifartrain()

epoch0, lossVariable containing:
 1.4387
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch1, lossVariable containing:
 0.9976
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch2, lossVariable containing:
 0.7389
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch3, lossVariable containing:
 0.4958
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch4, lossVariable containing:
 0.2694
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch5, lossVariable containing:
 0.1239
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch6, lossVariable containing:
1.00000e-02 *
  5.3022
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch7, lossVariable containing:
1.00000e-02 *
  2.3029
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch8, lossVariable containing:
1.00000e-02 *
  1.3496
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch9, lossVariable containing:
1.00000e-03 *
  8.2493
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Finished Training


In [15]:
# learning_rate = 3e-5
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# optimizer = Adam(net.parameters(), lr=learning_rate, amsgrad=True)
# LOSS, Test_LOSS= cifartrain()

epoch0, lossVariable containing:
 1.5133
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch1, lossVariable containing:
 1.1000
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch2, lossVariable containing:
 0.8518
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch3, lossVariable containing:
 0.6116
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch4, lossVariable containing:
 0.3754
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch5, lossVariable containing:
 0.1989
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch6, lossVariable containing:
1.00000e-02 *
  9.3962
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch7, lossVariable containing:
1.00000e-02 *
  4.7172
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch8, lossVariable containing:
1.00000e-02 *
  2.6213
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch9, lossVariable containing:
1.00000e-02 *
  1.6869
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Finished Training


### Longer Training

In [9]:
max_epoch=100

In [ ]:
learning_rate = 6e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
TEST_Acc = np.zeros([1, max_epoch])
optimizer = Adam(net.parameters(), lr=learning_rate, weight_decay=0.3, amsgrad=True)
LOSS, Test_LOSS, Test_Acc= cifartrain()


epoch0, lossVariable containing:
 1.5077
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 1.2225
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.003500
epoch1, lossVariable containing:
 1.0550
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.9866
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.001500
epoch2, lossVariable containing:
 0.8769
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.8400
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.003900
epoch3, lossVariable containing:
 0.7592
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test loss:Variable containing:
 0.7895
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Accuracy of the network on the 10000 test images: 0.023700
epoch4, lossVariable containing:
 0.6924
[torch.cuda.FloatTensor of size 1 (GPU 0)]

test lo

In [ ]:
sio.savemat('amsgrad_res_gen.mat',{'loss':LOSS, 'test_loss':Test_LOSS, 'test_acc':Test_Acc})

In [13]:
learning_rate = 6e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
optimizer = Adam(net.parameters(), lr=learning_rate, amsgrad=True)
LOSS, Test_LOSS= cifartrain()
sio.savemat('amsgrad_res.mat',{'loss':LOSS})

epoch0, lossVariable containing:
 1.3702
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch1, lossVariable containing:
 0.9068
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch2, lossVariable containing:
 0.6575
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch3, lossVariable containing:
 0.4340
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch4, lossVariable containing:
 0.2329
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch5, lossVariable containing:
 0.1203
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch6, lossVariable containing:
1.00000e-02 *
  5.4533
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch7, lossVariable containing:
1.00000e-02 *
  2.4356
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch8, lossVariable containing:
1.00000e-03 *
  9.6874
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch9, lossVariable containing:
1.00000e-03 *
  4.9833
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch10, lossVariable containing:
1.00000e-03 *
  3.4243
[torch.cuda.FloatTensor of size 1

epoch83, lossVariable containing:
1.00000e-04 *
  1.9236
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch84, lossVariable containing:
1.00000e-04 *
  1.9162
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch85, lossVariable containing:
1.00000e-04 *
  1.9710
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch86, lossVariable containing:
1.00000e-04 *
  1.9420
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch87, lossVariable containing:
1.00000e-04 *
  1.8586
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch88, lossVariable containing:
1.00000e-04 *
  1.8672
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch89, lossVariable containing:
1.00000e-04 *
  1.8335
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch90, lossVariable containing:
1.00000e-04 *
  1.7722
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch91, lossVariable containing:
1.00000e-04 *
  1.7807
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch92, lossVariable containing:
1.00000e-04 *
  1.7081
[torch.cuda.FloatTensor of size 1 

In [14]:
# learning_rate = 5e-5
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# optimizer = Adam(net.parameters(), lr=learning_rate, amsgrad=True)
# LOSS, Test_LOSS= cifartrain()
# sio.savemat('amsgrad_res1.mat',{'loss':LOSS})

epoch0, lossVariable containing:
 1.3858
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch1, lossVariable containing:
 0.9398
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch2, lossVariable containing:
 0.6760
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch3, lossVariable containing:
 0.4355
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch4, lossVariable containing:
 0.2204
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch5, lossVariable containing:
 0.1019
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch6, lossVariable containing:
1.00000e-02 *
  4.4211
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch7, lossVariable containing:
1.00000e-02 *
  1.9033
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch8, lossVariable containing:
1.00000e-03 *
  8.5368
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch9, lossVariable containing:
1.00000e-03 *
  5.4792
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch10, lossVariable containing:
1.00000e-03 *
  4.0852
[torch.cuda.FloatTensor of size 1

epoch83, lossVariable containing:
1.00000e-04 *
  2.5994
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch84, lossVariable containing:
1.00000e-04 *
  2.5262
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch85, lossVariable containing:
1.00000e-04 *
  2.5318
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch86, lossVariable containing:
1.00000e-04 *
  2.3552
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch87, lossVariable containing:
1.00000e-04 *
  2.6358
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch88, lossVariable containing:
1.00000e-04 *
  2.4442
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch89, lossVariable containing:
1.00000e-04 *
  2.3429
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch90, lossVariable containing:
1.00000e-04 *
  2.3568
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch91, lossVariable containing:
1.00000e-04 *
  2.3070
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch92, lossVariable containing:
1.00000e-04 *
  2.3768
[torch.cuda.FloatTensor of size 1 

In [17]:
# learning_rate = 6e-5
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# optimizer = Adam(net.parameters(), lr=learning_rate, betas=(0.9, 0.99), amsgrad=True)
# LOSS, Test_LOSS= cifartrain()
# sio.savemat('amsgrad_res2.mat',{'loss':LOSS})

epoch0, lossVariable containing:
 1.3698
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch1, lossVariable containing:
 0.9132
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch2, lossVariable containing:
 0.6540
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch3, lossVariable containing:
 0.4146
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch4, lossVariable containing:
 0.2094
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch5, lossVariable containing:
1.00000e-02 *
  9.4387
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch6, lossVariable containing:
1.00000e-02 *
  3.9940
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch7, lossVariable containing:
1.00000e-02 *
  1.7212
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch8, lossVariable containing:
1.00000e-03 *
  9.6116
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch9, lossVariable containing:
1.00000e-03 *
  6.2651
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch10, lossVariable containing:
1.00000e-03 *
  4.9588
[torch.cuda.FloatT

epoch82, lossVariable containing:
1.00000e-04 *
  3.2641
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch83, lossVariable containing:
1.00000e-04 *
  3.2866
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch84, lossVariable containing:
1.00000e-04 *
  3.0632
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch85, lossVariable containing:
1.00000e-04 *
  3.0229
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch86, lossVariable containing:
1.00000e-04 *
  3.0981
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch87, lossVariable containing:
1.00000e-04 *
  3.1734
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch88, lossVariable containing:
1.00000e-04 *
  3.0203
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch89, lossVariable containing:
1.00000e-04 *
  2.8576
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch90, lossVariable containing:
1.00000e-04 *
  3.0485
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch91, lossVariable containing:
1.00000e-04 *
  2.8847
[torch.cuda.FloatTensor of size 1 

In [18]:
learning_rate = 5.5e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, gamma=0.05)
LOSS, Test_LOSS= cifartrain()
sio.savemat('nosadam_res2.mat',{'loss':LOSS})

epoch0, lossVariable containing:
 1.3817
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch1, lossVariable containing:
 0.9280
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch2, lossVariable containing:
 0.6726
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch3, lossVariable containing:
 0.4438
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch4, lossVariable containing:
 0.2349
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch5, lossVariable containing:
 0.1129
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch6, lossVariable containing:
1.00000e-02 *
  5.3407
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch7, lossVariable containing:
1.00000e-02 *
  2.7391
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch8, lossVariable containing:
1.00000e-02 *
  1.4453
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch9, lossVariable containing:
1.00000e-03 *
  6.5580
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch10, lossVariable containing:
1.00000e-03 *
  3.6362
[torch.cuda.FloatTensor of size 1

epoch83, lossVariable containing:
1.00000e-05 *
  9.2639
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch84, lossVariable containing:
1.00000e-05 *
  9.1441
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch85, lossVariable containing:
1.00000e-05 *
  9.6740
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch86, lossVariable containing:
1.00000e-05 *
  8.6898
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch87, lossVariable containing:
1.00000e-05 *
  8.3940
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch88, lossVariable containing:
1.00000e-05 *
  8.4343
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch89, lossVariable containing:
1.00000e-05 *
  8.0498
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch90, lossVariable containing:
1.00000e-05 *
  8.0564
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch91, lossVariable containing:
1.00000e-05 *
  8.1853
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch92, lossVariable containing:
1.00000e-05 *
  7.5345
[torch.cuda.FloatTensor of size 1 

In [19]:
# learning_rate = 4.5e-5
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# optimizer = AdaStab(net.parameters(), lr=learning_rate, gamma=0.05)
# LOSS, Test_LOSS= cifartrain()
# sio.savemat('nosadam_res3.mat',{'loss':LOSS})

epoch0, lossVariable containing:
 1.4180
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch1, lossVariable containing:
 0.9779
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch2, lossVariable containing:
 0.7326
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch3, lossVariable containing:
 0.4975
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch4, lossVariable containing:
 0.2847
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch5, lossVariable containing:
 0.1392
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch6, lossVariable containing:
1.00000e-02 *
  6.2383
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch7, lossVariable containing:
1.00000e-02 *
  3.1329
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch8, lossVariable containing:
1.00000e-02 *
  1.4930
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch9, lossVariable containing:
1.00000e-02 *
  1.0283
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch10, lossVariable containing:
1.00000e-03 *
  5.3139
[torch.cuda.FloatTensor of size 1

KeyboardInterrupt: 

# Adam

In [8]:
max_epoch=10

In [9]:
learning_rate = 6e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
optimizer = Adam(net.parameters(), lr=learning_rate, amsgrad=False)
LOSS, Test_LOSS= cifartrain()
# sio.savemat('adam_res.mat',{'loss':LOSS})

epoch0, lossVariable containing:
 1.3776
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch1, lossVariable containing:
 0.9123
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch2, lossVariable containing:
 0.6554
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch3, lossVariable containing:
 0.4343
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch4, lossVariable containing:
 0.2325
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch5, lossVariable containing:
 0.1172
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch6, lossVariable containing:
1.00000e-02 *
  6.4895
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch7, lossVariable containing:
1.00000e-02 *
  4.8780
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch8, lossVariable containing:
1.00000e-02 *
  6.5994
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch9, lossVariable containing:
1.00000e-02 *
  7.0446
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Finished Training


In [10]:
learning_rate = 5e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
optimizer = Adam(net.parameters(), lr=learning_rate, amsgrad=False)
LOSS, Test_LOSS= cifartrain()
# sio.savemat('adam_res.mat',{'loss':LOSS})

KeyboardInterrupt: 

In [11]:
learning_rate = 4e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
optimizer = Adam(net.parameters(), lr=learning_rate, amsgrad=False)
LOSS, Test_LOSS= cifartrain()
# sio.savemat('adam_res.mat',{'loss':LOSS})

epoch0, lossVariable containing:
 1.4502
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch1, lossVariable containing:
 1.0165
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch2, lossVariable containing:
 0.7631
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch3, lossVariable containing:
 0.5216
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch4, lossVariable containing:
 0.2969
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch5, lossVariable containing:
 0.1413
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch6, lossVariable containing:
1.00000e-02 *
  6.9286
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch7, lossVariable containing:
1.00000e-02 *
  4.2577
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch8, lossVariable containing:
1.00000e-02 *
  3.5951
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch9, lossVariable containing:
1.00000e-02 *
  4.9119
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Finished Training


In [12]:
max_epoch=100

In [13]:
learning_rate = 4e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
optimizer = Adam(net.parameters(), lr=learning_rate, amsgrad=False)
LOSS, Test_LOSS= cifartrain()
sio.savemat('adam_res1.mat',{'loss':LOSS})

epoch0, lossVariable containing:
 1.4323
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch1, lossVariable containing:
 0.9892
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch2, lossVariable containing:
 0.7245
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch3, lossVariable containing:
 0.4780
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch4, lossVariable containing:
 0.2504
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch5, lossVariable containing:
 0.1129
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch6, lossVariable containing:
1.00000e-02 *
  5.0909
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch7, lossVariable containing:
1.00000e-02 *
  3.0962
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch8, lossVariable containing:
1.00000e-02 *
  4.1710
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch9, lossVariable containing:
1.00000e-02 *
  7.2261
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch10, lossVariable containing:
1.00000e-02 *
  4.8190
[torch.cuda.FloatTensor of size 1

epoch83, lossVariable containing:
1.00000e-03 *
  5.8893
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch84, lossVariable containing:
1.00000e-02 *
  1.6204
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch85, lossVariable containing:
1.00000e-02 *
  1.7327
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch86, lossVariable containing:
1.00000e-02 *
  1.2838
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch87, lossVariable containing:
1.00000e-03 *
  5.0789
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch88, lossVariable containing:
1.00000e-03 *
  2.4896
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch89, lossVariable containing:
1.00000e-03 *
  6.8553
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch90, lossVariable containing:
1.00000e-02 *
  2.2075
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch91, lossVariable containing:
1.00000e-02 *
  1.1914
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch92, lossVariable containing:
1.00000e-03 *
  7.8539
[torch.cuda.FloatTensor of size 1 

In [15]:
learning_rate = 4e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
optimizer = Adam(net.parameters(), lr=learning_rate,betas=(0.9, 0.99), amsgrad=False)
LOSS, Test_LOSS= cifartrain()
sio.savemat('adam_res2.mat',{'loss':LOSS})

epoch0, lossVariable containing:
 1.4254
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch1, lossVariable containing:
 0.9677
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch2, lossVariable containing:
 0.7123
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch3, lossVariable containing:
 0.4668
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch4, lossVariable containing:
 0.2510
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch5, lossVariable containing:
 0.1298
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch6, lossVariable containing:
1.00000e-02 *
  7.8508
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch7, lossVariable containing:
1.00000e-02 *
  6.2696
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch8, lossVariable containing:
1.00000e-02 *
  5.4074
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch9, lossVariable containing:
1.00000e-02 *
  4.7083
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch10, lossVariable containing:
1.00000e-02 *
  4.7992
[torch.cuda.FloatTensor of size 1

epoch83, lossVariable containing:
1.00000e-02 *
  1.0707
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch84, lossVariable containing:
1.00000e-03 *
  9.9261
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch85, lossVariable containing:
1.00000e-03 *
  8.5545
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch86, lossVariable containing:
1.00000e-02 *
  1.0604
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch87, lossVariable containing:
1.00000e-03 *
  9.6010
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch88, lossVariable containing:
1.00000e-03 *
  9.1138
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch89, lossVariable containing:
1.00000e-03 *
  9.7684
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch90, lossVariable containing:
1.00000e-02 *
  1.0246
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch91, lossVariable containing:
1.00000e-03 *
  9.9778
[torch.cuda.FloatTensor of size 1 (GPU 0)]

epoch92, lossVariable containing:
1.00000e-03 *
  8.9885
[torch.cuda.FloatTensor of size 1 

## AdaStab

In [ ]:
# learning_rate = 1e-4
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# optimizer = AdaStab(net.parameters(), lr=learning_rate, gamma=0.05)
# LOSS, Test_LOSS= cifartrain()

In [ ]:
# learning_rate = 2e-4
# net = ResNet18()
# net = net.cuda()
# criterion = nn.CrossEntropyLoss()
# LOSS = np.zeros([1, max_epoch])
# Test_LOSS = np.zeros([1, max_epoch])
# optimizer = AdaStab(net.parameters(), lr=learning_rate, gamma=0.05)
# LOSS, Test_LOSS= cifartrain()

In [ ]:
learning_rate = 5e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, gamma=0.05)
LOSS, Test_LOSS= cifartrain()

In [ ]:
learning_rate = 6e-5
net = ResNet18()
net = net.cuda()
criterion = nn.CrossEntropyLoss()
LOSS = np.zeros([1, max_epoch])
Test_LOSS = np.zeros([1, max_epoch])
optimizer = AdaStab(net.parameters(), lr=learning_rate, gamma=0.05)
LOSS, Test_LOSS= cifartrain()